In [2]:
from skt.gcp import (
    PROJECT_ID,
    bq_insert_overwrite   
)

In [3]:
from datetime import date, datetime, timedelta
import time
from datetime import date, datetime

import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from dateutil.relativedelta import relativedelta
from pyhive import hive

from copy import deepcopy
from joblib import Parallel, delayed
import warnings

from tqdm.notebook import tqdm

DT :  2024-06-11
DT_threshold :  2024-04-12
DT_threshold :  2024-06-04


In [ ]:
print(f'current_dt: {current_dt}')
print(f'state: {state}')

In [ ]:
PROJECT_ID = "skt-datahub"
data_db_name = "adot_reco"
save_db_name = "adot_reco_dev"
table_nm = "recgpt_log_sequence_daily_preprocessed_prd"
lag_table_nm = "recgpt_log_sequence_lag_daily_preprocessed_prd"

In [ ]:
tmap_temp_cat_list = {
    "교통편의": [],
    "생활편의": [],
    "쇼핑": [],
    "여행/레저": [],
    "의료편의": []
}

tmap_where_clauses = []
for key, vals in tmap_temp_cat_list.items():
    tmap_where_clauses.append(
        f"TRIM(LOWER(cat2)) = '{key}'"
    )
    tmap_where_clauses.append("OR")
tmap_where_clause = ' '.join(tmap_where_clauses[:-1])

In [ ]:
query = f"""
WITH TMAP_TABLE AS (
    SELECT  svc_mgmt_num,
            luna_id,
            "tmap" as source,
            TRIM(LOWER(cat2)) as cat2,
            TRIM(LOWER(cat1)) as cat1,
            TRIM(LOWER(item)) as item,
            unix_time,
            CASE 
                WHEN EXTRACT(DAYOFWEEK FROM dt) IN (1, 7) THEN 1
                ELSE 0
            END AS is_weekend,
            CASE
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 0 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 6 THEN '0_6'
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 6 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 10 THEN '6_9'
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 10 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 12 THEN '9_11'
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 12 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 13 THEN '11_13'
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 13 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 17 THEN '13_18'
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 17 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 21 THEN '18_21'
                WHEN EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) >= 21 
                    AND EXTRACT(HOUR FROM DATETIME(TIMESTAMP_SECONDS(CAST(unix_time AS INT64)), 'Asia/Seoul')) < 24 THEN '21_24'
            END AS second_s,
            dt

    FROM {data_db_name}.{table_nm}
    WHERE dt = '{current_dt}' and type = 'tmap' and ({tmap_where_clause})

)

SELECT  svc_mgmt_num,
        luna_id,
        item,
        cat1,
        cat2,
        count(*) as item_cnt,
        second_s,
        is_weekend,
        dt
FROM TMAP_TABLE
group by svc_mgmt_num, luna_id, item, cat1, cat2, dt, second_s, is_weekend
"""

In [ ]:
print(query)

In [ ]:
bq_insert_overwrite(sql=query, destination=f'{PROJECT_ID}.{save_db_name}.tmap_item_cnt', partition='dt')